# Test

## Test

In [1]:
import sys, os
import torch
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
import setting

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import scaler as ml_sclaer
import pathSetting

#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


### Test Data Preparation

### Get model meta by mongodb

In [5]:
from Clust.clust.ingestion.mongo.mongo_client import MongoClient
from Clust.setting import influx_setting_KETI as ins

mongo_client = MongoClient(ins.CLUSTMetaInfo2)

db_name = 'model'
collection_name = 'meta'

all_model_meta = mongo_client.get_all_document(db_name, collection_name)

ServerSelectionTimeoutError: 10.252.107.59:27170: timed out, Timeout: 30s, Topology Description: <TopologyDescription id: 642d2c08e145f4e41bb37a39, topology_type: Unknown, servers: [<ServerDescription ('10.252.107.59', 27170) server_type: Unknown, rtt: None, error=NetworkTimeout('10.252.107.59:27170: timed out')>]>

In [7]:
db_name = 'model'
collection_name = 'meta'
search = {'modelName': 'EnergyLSTMRegression'}
model_meta= mongo_client.get_document_by_json(db_name, collection_name, search)
print(model_meta)
model_meta = model_meta[0]

ServerSelectionTimeoutError: 10.252.107.59:27170: timed out, Timeout: 30s, Topology Description: <TopologyDescription id: 642d2b9d8aa51c55d1cd31a0, topology_type: Unknown, servers: [<ServerDescription ('10.252.107.59', 27170) server_type: Unknown, rtt: None, error=NetworkTimeout('10.252.107.59:27170: timed out')>]>

In [ ]:
# 2. select Model 
ModelMeta =p1.read_json_data(pathSetting.trainModelMetaFilePath)
modelList = list(ModelMeta.keys())
modelName = modelList[0]

windowNum=144

### Test data selection

In [2]:
# 1 (p2부분 1. Data Selection)
DataMeta = p1.read_json_data(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())

In [3]:
dataList

['IntegraionTrainX', 'IntegraionTrainy', 'IntegraionTestX', 'IntegraionTesty']

In [4]:
# 2.
# dataX
dataName_X = dataList[2]
data_save_mode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

# datay
dataName_y = dataList[3]
data_save_mode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]

# 3.
dataFolderName = "data_integrated_result"
current = os.getcwd()
dataFolderPath = os.path.join(current, dataFolderName)

data_X = ml_data.get_saved_integrated_data(data_save_mode_X, dataName_X, dataFolderPath)
data_y = ml_data.get_saved_integrated_data(data_save_mode_y, dataName_y, dataFolderPath)


### Test data preparation

In [7]:
# for regression
feature_list = model_meta["featureList"]
target = model_meta["target"]

model_file_path = model_meta['files']['modelFile']["filePath"]

model_method = model_meta["model_method"]

# model_parameter?
train_parameter = model_meta["trainParameter"]

# Scaling Test Input
X_scaler_file_path = model_meta['files']['XScalerFile']["filePath"]
y_scaler_file_path = model_meta['files']['yScalerFile']["filePath"]
scaler_param = model_meta["scalerParam"]
test_X, scaler_X = ml_sclaer.get_scaled_test_data(data_X[feature_list], X_scaler_file_path, scaler_param)
test_y, scaler_y = ml_sclaer.get_scaled_test_data(data_y[target], y_scaler_file_path, scaler_param)

In [ ]:
# transform

# regression
from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf

testX, testy = trans_df_to_np(test_X, test_y)

# forecast

In [ ]:
input_size, seq_len = testX.shape[1], testX.shape[2]
print(input_size, seq_len)

# RNN model parameters
modelParameter = {
    'rnn_type': 'gru',
    'input_size': input_size, 
    'hidden_size': 64,
    'num_layers': 2,
    'output_dim': 1, 
    'dropout': 0.1, 
    'bidirectional': True
}

# CNN_1D model parameters
# modelParameter = {
#     'input_size': input_size,
#     'seq_len': seq_len,
#     'output_channels': 64,
#     'kernel_size': 3,
#     'stride': 1,
#     'padding': 0, 
#     'dropout': 0.1
# }

# LSTM_FCNs model parameters
# modelParameter = {
#     'input_size': input_size,
#     'num_layers': 2,
#     'lstm_dropout': 0.4,
#     'fc_dropout': 0.1
# }

# FC model parameters
# modelParameter = {
#     'input_size': input_size,
#     'dropout': 0.1,
#     'bias': True
# }

testParameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': 'cpu', 
    'n_epochs': 10, 
    'batch_size': 16
}

#cleanParameter = 'NoClean'
model_method = 'GRU'

In [8]:
#3. Test model
from Clust.clust.ML.regression_YK.test import RegressionTest as RT

rt = RT()
rt.set_param(testParameter)
# print(train_parameter)
rt.set_model(model_method, model_file_path, modelParameter)
rt.set_data(testX, testy)
preds, trues = rt.test()

{'num_layers': 2, 'hidden_size': 64, 'dropout': 0.1, 'bidirectional': True, 'lr': 0.0001, 'device': 'cpu', 'n_epochs': 10, 'batch_size': 16, 'input_size': 24, 'seq_len': 144}

Start testing data

** Performance of test dataset ==> MSE = 0.0456344299018383, MAE = 0.15701907873153687
** Dimension of result for test dataset = (42,)


In [9]:
from Clust.clust.tool.stats_table import metrics

df_result = p4.get_prediction_df_result(preds, trues, scaler_param, scaler_y, feature_list= target, target_col = target[0])
result_metrics =  metrics.calculate_metrics_df(df_result)
print(df_result)
print(result_metrics)

scale
        value  prediction
0   15.370000   11.671662
1   17.059998   11.180050
2   10.120000   10.448577
3   14.410000   11.155736
4   11.570000   11.093349
5   17.799997   11.452437
6   13.620000   10.421119
7   12.439999   11.105028
8   10.680000   11.092004
9   15.580000   11.264921
10  15.120000   10.180723
11  13.469999   11.088959
12  10.990000   12.072914
13  15.590000   12.853612
14  24.120001   11.195452
15  20.650000   10.413429
16  10.890000   10.793626
17  17.370001   11.656283
18  13.480000   10.703239
19  11.990000   10.393258
20   9.680000   10.677022
21  13.760000   10.600958
22  14.889999   10.078066
23  14.970000   10.753571
24  20.929998   10.626743
25  20.879999   11.675137
26  12.140000   11.350747
27  12.320000   11.251084
28  14.560000   11.384274
29  10.470000   11.306628
30   9.330000   12.405560
31  10.690000   11.682135
32  12.950000   10.103333
33  15.060000    9.952890
34  13.429999   11.157265
35  10.599999   11.564406
36  11.420000   10.363724
37  11

In [13]:
from Clust.clust.tool.stats_table import metrics
result_metrics =  metrics.calculate_metrics_df(df_result)
print(result_metrics)

{'mae': 3.2879796, 'rmse': 4.473248331785011, 'r2': -0.6925636410364806, 'mape': 20.639748871326447}


In [14]:
# 5. Only For IPython Notebook
from Clust.clust.tool.plot.plot_two_data import plot_predictions
import plotly.offline as pyo
# Set notebook mode to work in offline
pyo.init_notebook_mode()
plot_predictions(df_result)

ModuleNotFoundError: No module named 'chart_studio'